In [1]:
import boto3
import pandas as pd
from io import StringIO, BytesIO
from datetime import datetime, timedelta

In [2]:
# adapter layer
def read_csv_to_df(bucket,key,decoding = 'utf-8', sep = ','):
    csv_obj = bucket.Object(key=key).get().get('Body').read().decode(decoding)
    data = StringIO(csv_obj)
    df = pd.read_csv(data, delimiter=sep)
    return df

def write_df_to_s3(bucket, df, key):
    out_buffer = BytesIO()
    df.to_parquet(out_buffer, index=False)  
    bucket.put_object(Body=out_buffer.getvalue(), Key=key)
    return True

def write_df_to_s3_csv(bucket, df, key):
    out_buffer = StringIO()
    df.to_csv(out_buffer, index=False)  
    bucket.put_object(Body=out_buffer.getvalue(), Key=key)
    return True
    
def list_files_in_prefix(bucket, prefix):
    files = [obj.key for obj in bucket.objects.filter(Prefix=prefix)]
    return files

In [3]:
# application layer

def extract(bucket, date_list):
    files = [key for date in date_list for key in list_files_in_prefix(bucket, date)]
    df = pd.concat([read_csv_to_df(bucket, obj) for obj in files[:15]], ignore_index=True)  
    return df

def transform_report1(df, columns, arg_date):
    df = df.loc[:, columns]
    df['opening_price'] = df.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('first')
    df['closing_price'] = df.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('last')
    df = df.groupby(['ISIN','Date'], as_index=False).agg(opening_price_eur=('opening_price', 'min'), closing_price_eur=('closing_price', 'min'),minimun_price_eur=('MinPrice', 'min'), maximun_price_eur=('MaxPrice', 'min'), daily_traded_volumen=('TradedVolume','sum')) 
    df['prev_closing_price'] = df.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_eur'].shift(1)
    df['change_prev_closing_%'] = (df['closing_price_eur'] -df['prev_closing_price']) / df ['prev_closing_price'] * 100
    df.drop(columns=['prev_closing_price'], inplace=True)
    df = df.round(decimals=2)
    #df = df[df.Date >= arg_date]
    return df

def load(bucket,df, trg_key, trg_format, meta_key, extract_date_list):
    key = trg_key + datetime.today().strftime("%Y%m%d_%H%M%S") + trg_format
    write_df_to_s3(bucket, df, key)
    update_meta_file(bucket, meta_key, extract_date_list)
    return True

def etl_report1(src_bucket, trg_bucket, date_list, columns, arg_date, trg_key, trg_format, meta_key):
   df = extract(src_bucket, date_list)
   df = transform_report1(df, columns, arg_date)
   extract_date_list = [date for date in date_list if date >= arg_date]
   load(trg_bucket,df, trg_key, trg_format, meta_key, extract_date_list)
   return True
    
    


In [4]:
# application layer not core

def return_date_list(bucket, arg_date, src_format, meta_key):
    min_date = datetime.strptime(arg_date, src_format).date() - timedelta(days=1)
    today = datetime.today().date()
    try:
        df_meta = read_csv_to_df(bucket, meta_key)
        dates = [(min_date + timedelta(days=x)) for x in range(0, (today - min_date).days + 1)]
        src_dates = set(pd.to_datetime(df_meta['source_date']).dt.date)
        dates_missing = set(dates[1:]) - src_dates
        if dates_missing:
            min_date = min(set(dates[1:]) - src_dates) - timedelta(days=1)
            return_dates = [date.strftime(src_format) for date in dates if date >= min_date]
            return_min_date = arg_date
        else:
            return_dates = []
            return_min_date = datetime(2200, 1, 1).date()
    except bucket.session.client('s3').exceptions.NoSuchKey:        
        return_dates = [(min_date + timedelta(days=x)).strftime(src_format) for x in range(0, (today - min_date).days + 1)]
        return_min_date = arg_date
    return return_min_date, return_dates 

def update_meta_file(bucket, meta_key, extract_date_list):
    df_new = pd.DataFrame(columns=['source_date', 'datetime_of_processing'])
    df_new['source_date'] = extract_date_list
    df_new['datetime_of_processing'] = datetime.today().strftime('%Y%m%d')
    df_old = read_csv_to_df(bucket,meta_key)
    df_all = pd.concat([df_old, df_new])
    write_df_to_s3_csv(bucket, df_all, meta_key)

In [5]:
# main funtion entrypoint
def main():
    # parameters /configurations
    #later read config
    arg_date = '2022-02-15'
    src_format = '%Y-%m-%d'
    src_bucket = 'xetra-1234'
    trg_bucket = 'josh-123456'
    columns=['ISIN', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice','EndPrice', 'TradedVolume']
    trg_key = 'xetra_daily_report_'
    trg_format = '.parquet'
    meta_key = "meta_file.csv" 

    # init
    s3 = boto3.resource('s3')
    bucket_src = s3.Bucket(src_bucket)
    bucket_trg = s3.Bucket(trg_bucket)

    # run application
    extract_date, date_list = return_date_list(bucket_trg, arg_date, src_format, meta_key)
    etl_report1(bucket_src, bucket_trg, date_list, columns, extract_date, trg_key, trg_format, meta_key)  

In [6]:
# run
main()

C:\Users\JOSUE\AppData\Local\Temp\ipykernel_14128\2240381619.py:5: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([read_csv_to_df(bucket, obj) for obj in files[:15]], ignore_index=True)


## READING THEUPLOAD FILE

In [7]:
trg_bucket = 'josh-123456'
s3 = boto3.resource('s3')
bucket_trg = s3.Bucket(trg_bucket)
for obj in bucket_trg.objects.all():
    print(obj.key)

meta_file.csv
xetra_daily_report_20240413_205242.parquet


In [8]:
prq_obj = bucket_trg.Object(key='xetra_daily_report_20240413_205242.parquet').get().get('Body').read()
data = BytesIO(prq_obj)
df_report = pd.read_parquet(data)

In [9]:
df_report.shape

(3053, 8)

In [10]:
df_report

,ISIN,Date,opening_price_eur,closing_price_eur,minimun_price_eur,maximun_price_eur,daily_traded_volumen,change_prev_closing_%
0,AT000000STR1,2022-02-14,39.30,38.55,38.00,38.00,299,NaN
1,AT00000FACC2,2022-02-14,8.94,8.52,8.52,8.52,3882,NaN
2,AT0000606306,2022-02-14,25.40,26.10,24.86,24.86,11875,NaN
3,AT0000644505,2022-02-14,104.40,104.60,104.00,104.00,134,NaN
4,AT0000652011,2022-02-14,42.15,41.92,41.11,41.11,1621,NaN
...,...,...,...,...,...,...,...,...
3048,XS2265370234,2022-02-14,20.35,20.73,20.35,20.35,519,NaN
3049,XS2284324667,2022-02-14,37.27,37.32,37.27,37.27,371,NaN
3050,XS2314659447,2022-02-14,8.69,8.69,8.69,8.69,0,NaN
3051,XS2314660700,2022-02-14,20.74,20.80,20.74,20.74,115,NaN
